# OpenMASTER - quickstart

-------------------------

In [1]:
import pyomo.environ as pyo
import openMASTER
from pyomo.core.base.suffix import Suffix

### Define the abstract model

In [2]:
m = openMASTER.make_model()

m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT_EXPORT)

### Model data upload

* If you haven't created the .csv files, please:
    * Be aware the openMASTER_Data.xlsx file has to be downloaded using git-lfs or the following link:
        https://github.com/IIT-EnergySystemModels/openMASTER/raw/main/data/input/openMASTER_Data.xlsx?download=
    * Run the first line of code in this cell, which will both create the .csv files and load them into the DataPortal (this whole function takes several minutes)

* On the contrary, if you have already created the .csv files from the Excel file and haven't changed them, you can directly go on to the second line of code. This will save some minutes.

In any case, add "#" in front of the line you are not running.

In [3]:
data = openMASTER.load_dataportal_from_excel('../data/input/openMASTER_Data.xlsx')
#data = openMASTER.load_dataportal_from_csv()

### Create the instance of the abstract model 

In [4]:
instance = m.create_instance(data)

In [5]:
'''
solver = pyo.SolverFactory('gurobi')

#solver.options['Method'] = 2  # Usa el método de barrera
solver.options['NumericFocus'] = 3  # Enfocar en la precisión numérica
solver.options['ScaleFlag'] = 2  # Escalado automático

#solver.options['DualReductions'] = 0  # Do not reduce the problem
#solver.options['IISMethod'] = 1  # Find IIS if infeasible
#solver.options['ResultFile'] = 'model.ilp'

solver_results = solver.solve(instance, keepfiles=False, tee=True)

instance.display()

#path        = "../data/input/openMASTER_Data.xlsx"
#output_path = '../data/tmp/output'
#sheetname   = "Output"
#d_vars_from_instance = openMASTER.export_model_to_csv(path, output_path, sheetname, instance)
'''

'\nsolver = pyo.SolverFactory(\'gurobi\')\n\n#solver.options[\'Method\'] = 2  # Usa el método de barrera\nsolver.options[\'NumericFocus\'] = 3  # Enfocar en la precisión numérica\nsolver.options[\'ScaleFlag\'] = 2  # Escalado automático\n\n#solver.options[\'DualReductions\'] = 0  # Do not reduce the problem\n#solver.options[\'IISMethod\'] = 1  # Find IIS if infeasible\n#solver.options[\'ResultFile\'] = \'model.ilp\'\n\nsolver_results = solver.solve(instance, keepfiles=False, tee=True)\n\ninstance.display()\n\n#path        = "../data/input/openMASTER_Data.xlsx"\n#output_path = \'../data/tmp/output\'\n#sheetname   = "Output"\n#d_vars_from_instance = openMASTER.export_model_to_csv(path, output_path, sheetname, instance)\n'

In [6]:
#import pyomo.contrib.iis
#pyomo.contrib.iis.write_iis(instance, "Infeasibilities.ilp", solver='gurobi')

In [7]:
import pandas as pd
import itertools
import os
import pyomo.environ as pyo
import time

# Define the relative path to the Excel file
relative_path = '../data/input/SD.xlsx'

# Read data from the Excel file
df_sd = pd.read_excel(relative_path, sheet_name='SD')

# Create unique lists for uncertainties and levels
uncertainties = df_sd['Uncertainty'].unique()
levels = ['Low', 'High']

# Create all combinations of levels for each uncertainty
scenarios = list(itertools.product(levels, repeat=len(uncertainties)))

# Create a list to store scenarios with their names
scenario_names = ['Scenario_' + str(i) for i in range(len(scenarios))]

# Create a DataFrame for scenarios
df_scenarios = pd.DataFrame(scenarios, columns=uncertainties, index=scenario_names)

# Save the scenario table to a CSV file
df_scenarios.to_csv('../data/output/scenarios_table.csv', index=True)

# Create an extended DataFrame
extended_data = []

# Initialize lists to store results
results_data = []

# Iterate over all combinations
for i, scenario in enumerate(scenarios):
    print(f"Iteration: {i}")
    print(f"Scenario: {scenario}")

    start_time = time.time()

    # Create a dictionary to store parameter values for the current scenario
    param_dict = {name: [] for name in df_sd['openMASTER parameter'].unique()}

    # Populate the dictionary with values of uncertainty and level combinations for the current scenario
    for unc, level in zip(uncertainties, scenario):
        subset = df_sd[(df_sd['Uncertainty'] == unc) & (df_sd['Level'] == level)]
        for param in subset['openMASTER parameter'].unique():
            param_values = subset[subset['openMASTER parameter'] == param]
            for _, row in param_values.iterrows():
                param_dict[param].append({
                    'value': row['Value'],
                    'element1': row['Parameter elements 1'] if pd.notna(row['Parameter elements 1']) else None,
                    'element2': row['Parameter elements 2'] if pd.notna(row['Parameter elements 2']) else None,
                    'year': row['Year'] if pd.notna(row['Year']) else None
                })

    # Assign values to parameters in the model instance
    for param, values in param_dict.items():
        for value_dict in values:
            #print(f"Setting {param} with elements ({value_dict['element1']}, {value_dict['element2']}, {value_dict['year']}) to value {value_dict['value']}")
            if value_dict['element1'] and value_dict['element2'] and value_dict['year']:
                instance.__getattribute__(param)[value_dict['element1'], value_dict['element2'], value_dict['year']] = value_dict['value']
            elif value_dict['element1'] and value_dict['element2']:
                instance.__getattribute__(param)[value_dict['element1'], value_dict['element2']] = value_dict['value']
            elif value_dict['element1'] and value_dict['year']:
                instance.__getattribute__(param)[value_dict['element1'], value_dict['year']] = value_dict['value']
            elif value_dict['element2'] and value_dict['year']:
                instance.__getattribute__(param)[value_dict['element2'], value_dict['year']] = value_dict['value']
            elif value_dict['element1']:
                instance.__getattribute__(param)[value_dict['element1']] = value_dict['value']
            elif value_dict['element2']:
                instance.__getattribute__(param)[value_dict['element2']] = value_dict['value']
            elif value_dict['year']:
                instance.__getattribute__(param)[value_dict['year']] = value_dict['value']
            else:
                instance.__getattribute__(param).set_value(value_dict['value'])

    solver = pyo.SolverFactory('gurobi')
    solver.options['NumericFocus'] = 3  # Focus on numerical precision
    solver.options['ScaleFlag'] = 2  # Automatic scaling
    
    solver_results = solver.solve(instance, keepfiles=False)
    
    # Check the solution status
    print(f"Solution status of iteration: {i}: {solver_results.solver.termination_condition}")

    # Export results
    path = "../data/input/openMASTER_Data.xlsx"
    base_folder = '../data/tmp/output'
    output_path = os.path.join(base_folder, str(i))
    sheetname = "Output"
    d_vars_from_instance = openMASTER.export_model_to_csv(path, output_path, sheetname, instance)

    # Retrieve model results

    # Calculate CO2 emissions
    CO2_emissions_2030 = instance.vEmiCO2Tot['y2030'].value
    CO2_budget_2030 = sum(instance.vEmiCO2Tot[year].value for year in ['y2020', 'y2025', 'y2030'])
    

    # Calculate fossil fuel use in 2030
    total_fossil = sum(instance.vQPEDom[sPE, 'y2030', sSeason, sDay, sHour].value for sPE in ['sPEIMPCO', 'sPENAGAS', 'sPELNGAS', 'sPECROIL']
                       for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour) + \
                   sum(instance.vQPEImp[sPE, 'y2030', sSeason, sDay, sHour].value for sPE in ['sPEIMPCO', 'sPENAGAS', 'sPELNGAS', 'sPECROIL']
                       for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour)
    total_primary_energy = sum(instance.vQPEDom[sPE, 'y2030', sSeason, sDay, sHour].value for sPE in instance.sPE
                               for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour) + \
                           sum(instance.vQPEImp[sPE, 'y2030', sSeason, sDay, sHour].value for sPE in instance.sPE
                               for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour)
    fossil_fuel_use_2030 = total_fossil
    fossil_fuel_mix_2030 = total_fossil / total_primary_energy
    

    # Calculate RES deployment in 2030
    renewable_sources = ['sCEHYRURIV', 'sCEHYRSCAP', 'sCEMINIHYDR', 'sCEWINDON', 'sCEWINDOFF',
                        'sCESOPHVCEWT', 'sCESOPHVDIWOTIND', 'sCESOPHVDIWOTOTH', 'sCESOTELCE',
                     'sCEBIOELECE']

    total_renewable_generation = (sum(instance.vQCEPriOUT[sCE, sTE, 'y2030', sSeason, sDay, sHour].value 
                                     for (sCE, sTE) in instance.sQCEPriOUT if sCE in renewable_sources
                                     for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour)
                                + sum(instance.vQCESecOUT[sCE, sTE, 'y2030', sSeason, sDay, sHour].value
                                     for (sCE, sTE) in instance.sQCESecOUT if sCE in renewable_sources
                                     for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour))

    total_electricity_generation = (sum(instance.vQCEPriOUT[sCE, sTE, 'y2030', sSeason, sDay, sHour].value 
                                     for (sCE, sTE) in instance.sQCEPriOUT
                                     for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour)
                                + sum(instance.vQCESecOUT[sCE, sTE, 'y2030', sSeason, sDay, sHour].value
                                     for (sCE, sTE) in instance.sQCESecOUT
                                     for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour))

    RES_mix_2030 = total_renewable_generation / total_electricity_generation
    total_renewable_generation_2030 = total_renewable_generation
    total_electricity_generation_2030 = total_electricity_generation


    # Calculate electrification in 2030
    electricity_vectors = ['sTEELEIND', 'sTEELEOTH', 'sTEELETRA']

    electricity_consumed = sum(instance.vQSTInTE[sTE, sST, sES, sVin, 'y2030', sSeason, sDay, sHour].value
                                   for (sTE,sST,sES) in instance.sQTESTES_Ele for sVin in instance.sVin if (sVin,'y2030') in instance.sVinYear
                                   for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour)

    total_energy_consumed = sum(instance.vQSTInTE[sTE, sST, sES, sVin, 'y2030', sSeason, sDay, sHour].value
                                   for (sTE,sST,sES) in instance.sQTESTES for sVin in instance.sVin if (sVin,'y2030') in instance.sVinYear
                                   for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour)

    electrification_2030 = electricity_consumed / total_energy_consumed


        # Calculate EV adoption rate in 2030

    EV_vectors = ['sST_DSTRA_LNP_CABEV', 'sST_DSTRA_LNP_CAGSNPIHYB', 'sST_DSTRA_LNP_CADIEPIHYB']
    Car_vectors = ['sST_DSTRA_LNP_CAGSN', 'sST_DSTRA_LNP_CADST', 'sST_DSTRA_LNP_CACNG',
                   'sST_DSTRA_LNP_CALPG', 'sST_DSTRA_LNP_CABIOE85', 'sST_DSTRA_LNP_CABIOD85',
                   'sST_DSTRA_LNP_CAGSNPIHYB', 'sST_DSTRA_LNP_CADIEPIHYB', 'sST_DSTRA_LNP_CABEV']

    EV_2030 = sum(instance.vSTTotCap[sST, sVin, 'y2030'].value for sVin in instance.sVin if (sVin,'y2030') in instance.sVinYear
                      for sST in EV_vectors)
    Tot_Car_2030 = sum(instance.vSTTotCap[sST, sVin, 'y2030'].value for sVin in instance.sVin if (sVin,'y2030') in instance.sVinYear
                           for sST in Car_vectors)

    EV_2030 = sum(instance.vSTTotCap[sST, sVin, 'y2030'].value for sVin in instance.sVin if (sVin,'y2030') in instance.sVinYear
                      for sST in EV_vectors)
    Tot_Car_2030 = sum(instance.vSTTotCap[sST, sVin, 'y2030'].value for sVin in instance.sVin if (sVin,'y2030') in instance.sVinYear
                           for sST in Car_vectors)

        # Avoid division by zero
    if Tot_Car_2030 > 0:
            EV_rate_2030 = EV_2030 / Tot_Car_2030
    else:
            EV_rate_2030 = 0.0  # If no conventional vehicles are present, adoption rate is zero


    if Tot_Car_2030 > 0:
            EV_rate_2030 = EV_2030 / Tot_Car_2030
    else:
            EV_rate_2030 = 0.0  # If no conventional vehicles are present, adoption rate is zero



        # Energy transition cost
    acc_total_cost = sum(instance.vTotalCost[sYear].value for sYear in ['y2020', 'y2025', 'y2030'])
    energy_transition_cost = acc_total_cost

        # Total cost in 2030
    total_cost_2030 = instance.vTotalCost['y2030'].value

        # Energy dependency in 2030
    total_energy_imported = sum(instance.vQPEImp[sPE, 'y2030', sSeason, sDay, sHour].value for sPE in instance.sPE
                                    for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour)
    total_energy_domestic = sum(instance.vQPEDom[sPE, 'y2030', sSeason, sDay, sHour].value for sPE in instance.sPE
                                   for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour)
    energy_dependency_2030 = total_energy_imported / (total_energy_imported + total_energy_domestic)

    
        # Electricity affordability in 2030
        # Create lists to store dual values and consumptions
    dual_values = []
    consumptions = []

        # Iterate over the relevant elements of sTE

    for sSeason in instance.sSeason:
        for sDay in instance.sDay:
            for sHour in instance.sHour:
                try:
                    dual_value = instance.dual[instance.EQ_TEBalance['sTEELECE', 'y2030', sSeason, sDay, sHour]]
                    consumption = sum(instance.vQSTInTE[sTE, sST, sES, sVin, 'y2030', sSeason, sDay, sHour].value
                                      for (sTE, sST, sES) in instance.sQTESTES_Ele if sTE in electricity_vectors
                                      for sVin in instance.sVin if (sVin, 'y2030') in instance.sVinYear)
                    dual_values.append(dual_value)
                    consumptions.append(consumption)
                except KeyError:
                    print(f"No dual value for EQ_TEBalance['sTEELECE', 'y2030', {sSeason}, {sDay}, {sHour}]")

        # Calculate the weighted average of the dual values
    total_consumption = sum(consumptions)
    weighted_sum_dual_values = sum(dual * consumption for dual, consumption in zip(dual_values, consumptions))
    electricity_affordability_2030 = weighted_sum_dual_values / total_consumption if total_consumption != 0 else 0
    

    results_data.append({
    'Scenario': scenario_names[i],
    'CO2_emissions_2030': CO2_emissions_2030,
    'CO2_budget_2030': CO2_budget_2030,
    'fossil_fuel_use_2030': fossil_fuel_use_2030,
    'fossil_fuel_mix_2030': fossil_fuel_mix_2030,
    'RES_mix_2030': RES_mix_2030,
    'total_renewable_generation_2030': total_renewable_generation_2030,
    'total_electricity_generation_2030': total_electricity_generation_2030,
    'electrification_2030': electrification_2030,
    'EV_rate_2030': EV_rate_2030,
    'EV_rate_2030': EV_rate_2030,
    'energy_transition_cost': energy_transition_cost,
    'total_cost_2030': total_cost_2030,
    'electricity_affordability_2030': electricity_affordability_2030,
    'energy_dependency_2030': energy_dependency_2030
    })

    # Imprimir resultados parciales para cada iteración
    print(f"CO2 emissions in 2030 for scenario {i}: {CO2_emissions_2030}")
    print(f"Energy_transition_cost {i}: {energy_transition_cost}")
    print(f"RES mix in 2030 for scenario {i}: {RES_mix_2030}")
    print(f"Electricity price in 2030 for scenario {i}: {electricity_affordability_2030}")


# Save results to a CSV file
results_df = pd.DataFrame(results_data)
results_df.to_csv('../data/output/scenarios_results.csv', index=False)

Iteration: 0
Scenario: ('Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low')
Solution status of iteration: 0: optimal
CO2 emissions in 2030 for scenario 0: 152.6618947168085
Energy_transition_cost 0: 783.6592946311965
RES mix in 2030 for scenario 0: 0.08906821228266723
Electricity price in 2030 for scenario 0: 0.0002488002303471269
Iteration: 1
Scenario: ('Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'High')
        dual
    LP writer cannot export suffixes to LP files.  Skipping.
Solution status of iteration: 1: optimal
CO2 emissions in 2030 for scenario 1: 153.07680987399505
Energy_transition_cost 1: 778.5121491009551
RES mix in 2030 for scenario 1: 0.08915627522982365
Electricity price in 2030 for scenario 1: 0.00024910300381615917
Iteration: 2
Scenario: ('Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'High', 'Low')
        dual
    LP writer cannot export suffixes to LP files.  Skipping.
Solution status of iteration: 2: optimal
CO2 emissions in 2030 for scenar

### Solve the model instance

To solve the model instance, please select a solver within the Pyomo SolverFactory. Please note that any solver has to be previously installed.

In [ ]:
import pandas as pd
import itertools
import os
import pyomo.environ as pyo
import time

# Define the relative path to the Excel file
relative_path = '../data/input/SD.xlsx'

# Read data from the Excel file
df_sd = pd.read_excel(relative_path, sheet_name='SD')

# Create unique lists for uncertainties and levels
uncertainties = df_sd['Uncertainty'].unique()
levels = ['Low', 'High']

# Create all combinations of levels for each uncertainty
scenarios = list(itertools.product(levels, repeat=len(uncertainties)))

# Create a list to store scenarios with their names
scenario_names = ['Scenario_' + str(i) for i in range(len(scenarios))]

# Create a DataFrame for scenarios
df_scenarios = pd.DataFrame(scenarios, columns=uncertainties, index=scenario_names)

# Save the scenario table to a CSV file
df_scenarios.to_csv('../data/output/scenarios_table.csv', index=True)

# Create an extended table that includes assigned values
extended_scenarios = []

# Create a dictionary to store parameter values for each scenario
param_dict = {name: [] for name in df_sd['openMASTER parameter'].unique()}

# Populate the dictionary with values of uncertainty and level combinations
for scenario in scenarios:
    scenario_values = {unc: {} for unc in uncertainties}
    for unc, level in zip(uncertainties, scenario):
        subset = df_sd[(df_sd['Uncertainty'] == unc) & (df_sd['Level'] == level)]
        for param in subset['openMASTER parameter'].unique():
            param_values = subset[subset['openMASTER parameter'] == param]
            for _, row in param_values.iterrows():
                param_dict[param].append({
                    'value': row['Value'],
                    'element1': row['Parameter elements 1'] if pd.notna(row['Parameter elements 1']) else None,
                    'element2': row['Parameter elements 2'] if pd.notna(row['Parameter elements 2']) else None,
                    'year': row['Year'] if pd.notna(row['Year']) else None
                })
                key = (param, row['Parameter elements 1'], row['Parameter elements 2'], row['Year'])
                scenario_values[unc][key] = {'value': row['Value'], 'level': level}
    extended_scenarios.append(scenario_values)



# Create an extended DataFrame
extended_data = []

for scenario_name, scenario_values in zip(scenario_names, extended_scenarios):
    for unc, values in scenario_values.items():
        for key, value in values.items():
            param, element1, element2, year = key
            extended_data.append({
                'Scenario': scenario_name,
                'Uncertainty': unc,
                'Parameter': param,
                'Element1': element1,
                'Element2': element2,
                'Year': year,
                'Level': value['level'],
                'Value': value['value']
                
            })

df_extended_scenarios = pd.DataFrame(extended_data)

# Save the extended table to a CSV file
df_extended_scenarios.to_csv('../data/output/extended_scenarios_table.csv', index=False)

# Print the scenario table
print(df_scenarios)


In [ ]:

# Initialize lists to store results

results_data = []


# Iterate over all combinations
for i, scenario in enumerate(scenarios):
    print(f"Iteration: {i}")
    print(f"Scenario: {scenario}")

    start_time = time.time()

    # Assign values to parameters in the model instance
    for param, values in param_dict.items():
        for value_dict in values:
            print(f"Setting {param} with elements ({value_dict['element1']}, {value_dict['element2']}, {value_dict['year']}) to value {value_dict['value']}")
            if value_dict['element1'] and value_dict['element2'] and value_dict['year']:
                instance.__getattribute__(param)[value_dict['element1'], value_dict['element2'], value_dict['year']] = value_dict['value']
            elif value_dict['element1'] and value_dict['element2']:
                instance.__getattribute__(param)[value_dict['element1'], value_dict['element2']] = value_dict['value']
            elif value_dict['element1'] and value_dict['year']:
                instance.__getattribute__(param)[value_dict['element1'], value_dict['year']] = value_dict['value']
            elif value_dict['element2'] and value_dict['year']:
                instance.__getattribute__(param)[value_dict['element2'], value_dict['year']] = value_dict['value']
            elif value_dict['element1']:
                instance.__getattribute__(param)[value_dict['element1']] = value_dict['value']
            elif value_dict['element2']:
                instance.__getattribute__(param)[value_dict['element2']] = value_dict['value']
            elif value_dict['year']:
                instance.__getattribute__(param)[value_dict['year']] = value_dict['value']
            else:
                instance.__getattribute__(param).set_value(value_dict['value'])

    solver = pyo.SolverFactory('gurobi')
    solver.options['NumericFocus'] = 3  # Focus on numerical precision
    solver.options['ScaleFlag'] = 2  # Automatic scaling
    
    solver_results = solver.solve(instance, keepfiles=False)
    
    # Check the solution status
    print(f"Solution status of iteration: {i}: {solver_results.solver.termination_condition}")

    # Elimina el componente dual existente si ya está definido
    if 'dual' in instance.component_map(Suffix):
        instance.del_component(instance.dual)

    # Agrega el sufijo dual para obtener valores duales
    instance.dual = Suffix(direction=Suffix.IMPORT)

    # Export results
    path = "../data/input/openMASTER_Data.xlsx"
    base_folder = '../data/tmp/output'
    output_path = os.path.join(base_folder, str(i))
    sheetname = "Output"
    d_vars_from_instance = openMASTER.export_model_to_csv(path, output_path, sheetname, instance)

    # Retrieve model results

    # Calculate CO2 emissions
    CO2_emissions_2030 = instance.vEmiCO2Tot['y2030'].value
    CO2_budget_2030 = sum(instance.vEmiCO2Tot[year].value for year in ['y2020', 'y2025', 'y2030'])
    

    # Calculate fossil fuel use in 2030
    total_fossil = sum(instance.vQPEDom[sPE, 'y2030', sSeason, sDay, sHour].value for sPE in ['sPEIMPCO', 'sPENAGAS', 'sPELNGAS', 'sPECROIL']
                       for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour) + \
                   sum(instance.vQPEImp[sPE, 'y2030', sSeason, sDay, sHour].value for sPE in ['sPEIMPCO', 'sPENAGAS', 'sPELNGAS', 'sPECROIL']
                       for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour)
    total_primary_energy = sum(instance.vQPEDom[sPE, 'y2030', sSeason, sDay, sHour].value for sPE in instance.sPE
                               for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour) + \
                           sum(instance.vQPEImp[sPE, 'y2030', sSeason, sDay, sHour].value for sPE in instance.sPE
                               for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour)
    fossil_fuel_use_2030 = total_fossil
    fossil_fuel_mix_2030 = total_fossil / total_primary_energy
    

    # Calculate RES deployment in 2030
    renewable_sources = ['sCEHYRURIV', 'sCEHYRSCAP', 'sCEMINIHYDR', 'sCEWINDON', 'sCEWINDOFF',
                        'sCESOPHVCEWT', 'sCESOPHVDIWOTIND', 'sCESOPHVDIWOTOTH', 'sCESOTELCE',
                     'sCEBIOELECE']

    total_renewable_generation = (sum(instance.vQCEPriOUT[sCE, sTE, 'y2030', sSeason, sDay, sHour].value 
                                     for (sCE, sTE) in instance.sQCEPriOUT if sCE in renewable_sources
                                     for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour)
                                + sum(instance.vQCESecOUT[sCE, sTE, 'y2030', sSeason, sDay, sHour].value
                                     for (sCE, sTE) in instance.sQCESecOUT if sCE in renewable_sources
                                     for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour))

    total_electricity_generation = (sum(instance.vQCEPriOUT[sCE, sTE, 'y2030', sSeason, sDay, sHour].value 
                                     for (sCE, sTE) in instance.sQCEPriOUT
                                     for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour)
                                + sum(instance.vQCESecOUT[sCE, sTE, 'y2030', sSeason, sDay, sHour].value
                                     for (sCE, sTE) in instance.sQCESecOUT
                                     for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour))

    RES_mix_2030 = total_renewable_generation / total_electricity_generation
    total_renewable_generation_2030 = total_renewable_generation
    total_electricity_generation_2030 = total_electricity_generation


    # Calculate electrification in 2030
    electricity_vectors = ['sTEELEIND', 'sTEELEOTH', 'sTEELETRA']

    electricity_consumed = sum(instance.vQSTInTE[sTE, sST, sES, sVin, 'y2030', sSeason, sDay, sHour].value
                                   for (sTE,sST,sES) in instance.sQTESTES_Ele for sVin in instance.sVin if (sVin,'y2030') in instance.sVinYear
                                   for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour)

    total_energy_consumed = sum(instance.vQSTInTE[sTE, sST, sES, sVin, 'y2030', sSeason, sDay, sHour].value
                                   for (sTE,sST,sES) in instance.sQTESTES for sVin in instance.sVin if (sVin,'y2030') in instance.sVinYear
                                   for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour)

    electrification_2030 = electricity_consumed / total_energy_consumed


        # Calculate EV adoption rate in 2030

    EV_vectors = ['sST_DSTRA_LNP_CABEV', 'sST_DSTRA_LNP_CAGSNPIHYB', 'sST_DSTRA_LNP_CADIEPIHYB']
    Car_vectors = ['sST_DSTRA_LNP_CAGSN', 'sST_DSTRA_LNP_CADST', 'sST_DSTRA_LNP_CACNG',
                   'sST_DSTRA_LNP_CALPG', 'sST_DSTRA_LNP_CABIOE85', 'sST_DSTRA_LNP_CABIOD85',
                   'sST_DSTRA_LNP_CAGSNPIHYB', 'sST_DSTRA_LNP_CADIEPIHYB', 'sST_DSTRA_LNP_CABEV']

    EV_2030 = sum(instance.vSTTotCap[sST, sVin, 'y2030'].value for sVin in instance.sVin if (sVin,'y2030') in instance.sVinYear
                      for sST in EV_vectors)
    Tot_Car_2030 = sum(instance.vSTTotCap[sST, sVin, 'y2030'].value for sVin in instance.sVin if (sVin,'y2030') in instance.sVinYear
                           for sST in Car_vectors)

    EV_2030 = sum(instance.vSTTotCap[sST, sVin, 'y2030'].value for sVin in instance.sVin if (sVin,'y2030') in instance.sVinYear
                      for sST in EV_vectors)
    Tot_Car_2030 = sum(instance.vSTTotCap[sST, sVin, 'y2030'].value for sVin in instance.sVin if (sVin,'y2030') in instance.sVinYear
                           for sST in Car_vectors)

        # Avoid division by zero
    if Tot_Car_2030 > 0:
            EV_rate_2030 = EV_2030 / Tot_Car_2030
    else:
            EV_rate_2030 = 0.0  # If no conventional vehicles are present, adoption rate is zero


    if Tot_Car_2030 > 0:
            EV_rate_2030 = EV_2030 / Tot_Car_2030
    else:
            EV_rate_2030 = 0.0  # If no conventional vehicles are present, adoption rate is zero



        # Energy transition cost
    acc_total_cost = sum(instance.vTotalCost[sYear].value for sYear in ['y2020', 'y2025', 'y2030'])
    energy_transition_cost = acc_total_cost

        # Total cost in 2030
    total_cost_2030 = instance.vTotalCost['y2030'].value

        # Energy dependency in 2030
    total_energy_imported = sum(instance.vQPEImp[sPE, 'y2030', sSeason, sDay, sHour].value for sPE in instance.sPE
                                    for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour)
    total_energy_domestic = sum(instance.vQPEDom[sPE, 'y2030', sSeason, sDay, sHour].value for sPE in instance.sPE
                                   for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour)
    energy_dependency_2030 = total_energy_imported / (total_energy_imported + total_energy_domestic)


        # Electricity affordability in 2030
        # Create lists to store dual values and consumptions
    dual_values = []
    consumptions = []

        # Iterate over the relevant elements of sTE

    for sSeason in instance.sSeason:
        for sDay in instance.sDay:
            for sHour in instance.sHour:
                try:
                    dual_value = instance.dual[instance.EQ_TEBalance['sTEELECE', 'y2030', sSeason, sDay, sHour]]
                    consumption = sum(instance.vQSTInTE[sTE, sST, sES, sVin, 'y2030', sSeason, sDay, sHour].value
                                      for (sTE, sST, sES) in instance.sQTESTES_Ele if sTE in ['sTEELECE']
                                      for sVin in instance.sVin if (sVin, 'y2030') in instance.sVinYear)
                    dual_values.append(dual_value)
                    consumptions.append(consumption)
                except KeyError:
                    print(f"No dual value for EQ_TEBalance['sTEELECE', 'y2030', {sSeason}, {sDay}, {sHour}]")

        # Calculate the weighted average of the dual values
    total_consumption = sum(consumptions)
    weighted_sum_dual_values = sum(dual * consumption for dual, consumption in zip(dual_values, consumptions))
    electricity_affordability_2030 = weighted_sum_dual_values / total_consumption if total_consumption != 0 else 0


    scenario_data = {
    'CO2_emissions_2030': CO2_emissions_2030,
    'CO2_budget_2030': CO2_budget_2030,
    'fossil_fuel_use_2030': fossil_fuel_use_2030,
    'fossil_fuel_mix_2030': fossil_fuel_mix_2030,
    'RES_mix_2030': RES_mix_2030,
    'total_renewable_generation_2030': total_renewable_generation_2030,
    'total_electricity_generation_2030': total_electricity_generation_2030,
    'electrification_2030': electrification_2030,
    'EV_rate_2030': EV_rate_2030,
    'EV_rate_2030': EV_rate_2030,
    'energy_transition_cost': energy_transition_cost,
    'total_cost_2030': total_cost_2030,
    'electricity_affordability_2030': electricity_affordability_2030,
    'energy_dependency_2030': energy_dependency_2030
    }

    # Imprimir resultados parciales para cada iteración
    print(f"CO2 emissions in 2030 for scenario {i}: {CO2_emissions_2030}")
    print(f"Energy_transition_cost {i}: {energy_transition_cost}")
    print(f"RES mix in 2030 for scenario {i}: {RES_mix_2030}")

    # Add parameter values to the data
    for unc, level in zip(uncertainties, scenario):
        scenario_data[unc] = level

    # Add parameter values to the scenario data
    for param, values in param_dict.items():
        for value_dict in values:
            param_name = f"{param}_{value_dict['element1']}_{value_dict['element2']}_{value_dict['year']}"
            scenario_data[param_name] = value_dict['value']
            
    # Append the scenario data to the results list
    results_data.append(scenario_data)

    # Create a DataFrame for the results
    df_results = pd.DataFrame(results_data)

    # Save the results to a CSV file
    df_results.to_csv('../data/output/model_results.csv', index=False)


    end_time = time.time()
    iteration_time = end_time - start_time
    print(f"Iteration Time: {iteration_time} seconds")

# Create a DataFrame for the results
df_results = pd.DataFrame(results_data)

# Save the results to a CSV file
df_results.to_csv('../data/output/results_SD.csv', index=False)


In [ ]:
'''
import pandas as pd
import itertools

# Define parameter names and their possible values
param_names = [
    'CO2_cost', 
    'Coal_cost', 'NG_cost', 'LNG_cost', 'Oil_cost', 
    'Wind_MaxRES', 'Solar_MaxRES', 
    'Wind_capex',
    'Solar_capex',
    'EV_capex',
    'Urban_mob_dem', 
    'Car_occup_rate',
    'Eff_buildings_HE', 'Eff_buildings_LE'
]

# Define the possible values for each parameter

#min: nominal value, max: 50% increase from nominal value
param_values = [
    [100, 150],                                      # CO2_cost pEmiCO2Cost
    [(8, 18.4 , 37, 40), (12, 27.6 , 55.5 , 60)],    # Coal_cost, NG_cost, LNG_cost, Oil_cost pPECost
    [(50, 50), (100, 100)],                          # Wind_MaxRES, Solar_MaxRES pCEMaxCap
    [1000, 2000],                                    # Wind_capex pCECapex
    [300, 800],                                      # Solar_capex pCECapex
    [20000, 50000],                                  # EV_capex pSTCapex
    [3500, 4000],                                    # Urban_mob_dem pDC
    [1.1, 1.5],                                      # Occup rate pAFTra
    [(0.16, 0.86), (0.25, 0.75)]                     # Eff_buildings_HE, Eff_buildings_LE pDC
]

# Generate all combinations of parameter values
param_combinations = list(itertools.product(*param_values))

# Create a dictionary to store the parameter names and values
param_dict = {name: [] for name in param_names}

# Populate the dictionary with values from combinations
for comb in param_combinations:
    param_dict['CO2_cost'].append(comb[0])
    param_dict['Coal_cost'].append(comb[1][0])
    param_dict['NG_cost'].append(comb[1][1])
    param_dict['LNG_cost'].append(comb[1][2])
    param_dict['Oil_cost'].append(comb[1][3])
    param_dict['Wind_MaxRES'].append(comb[2][0])
    param_dict['Solar_MaxRES'].append(comb[2][1])
    param_dict['Wind_capex'].append(comb[3])
    param_dict['Solar_capex'].append(comb[4])
    param_dict['EV_capex'].append(comb[5])
    param_dict['Urban_mob_dem'].append(comb[6])
    param_dict['Car_occup_rate'].append(comb[7])
    param_dict['Eff_buildings_HE'].append(comb[8][0])
    param_dict['Eff_buildings_LE'].append(comb[8][1])

# Create a DataFrame from the dictionary
df = pd.DataFrame(param_dict)

# Print the first few rows of the DataFrame
print(df)
'''

In [ ]:

# Initialize lists to store results

results_data = []


# Iterate over all combinations
for i, scenario in enumerate(scenarios):
    print(f"Iteration: {i}")
    start_time = time.time()

    # Assign values to parameters in the model instance
    for param, values in param_dict.items():
        for value_dict in values:
            if value_dict['element1'] and value_dict['element2'] and value_dict['year']:
                instance.__getattribute__(param)[value_dict['element1'], value_dict['element2'], value_dict['year']] = value_dict['value']
            elif value_dict['element1'] and value_dict['element2']:
                instance.__getattribute__(param)[value_dict['element1'], value_dict['element2']] = value_dict['value']
            elif value_dict['element1'] and value_dict['year']:
                instance.__getattribute__(param)[value_dict['element1'], value_dict['year']] = value_dict['value']
            elif value_dict['element2'] and value_dict['year']:
                instance.__getattribute__(param)[value_dict['element2'], value_dict['year']] = value_dict['value']
            elif value_dict['element1']:
                instance.__getattribute__(param)[value_dict['element1']] = value_dict['value']
            elif value_dict['element2']:
                instance.__getattribute__(param)[value_dict['element2']] = value_dict['value']
            elif value_dict['year']:
                instance.__getattribute__(param)[value_dict['year']] = value_dict['value']
            else:
                instance.__getattribute__(param).set_value(value_dict['value'])

    solver = pyo.SolverFactory('gurobi')
    solver.options['NumericFocus'] = 3  # Focus on numerical precision
    solver.options['ScaleFlag'] = 2  # Automatic scaling
    
    solver_results = solver.solve(instance, keepfiles=False, tee=True)
    
    # Check the solution status
    print(f"Solution status of iteration: {i}: {solver_results.solver.termination_condition}")

    # Elimina el componente dual existente si ya está definido
    if 'dual' in instance.component_map(Suffix):
        instance.del_component(instance.dual)

    # Agrega el sufijo dual para obtener valores duales
    instance.dual = Suffix(direction=Suffix.IMPORT)

    # Export results
    path = "../data/input/openMASTER_Data.xlsx"
    base_folder = '../data/tmp/output'
    output_path = os.path.join(base_folder, str(i))
    sheetname = "Output"
    d_vars_from_instance = openMASTER.export_model_to_csv(path, output_path, sheetname, instance)

    # Retrieve model results

    # Calculate CO2 emissions
    CO2_emissions_2030 = instance.vEmiCO2Tot['y2030'].value
    CO2_budget_2030 = sum(instance.vEmiCO2Tot[year].value for year in ['y2020', 'y2025', 'y2030'])
    

    # Calculate fossil fuel use in 2030
    total_fossil = sum(instance.vQPEDom[sPE, 'y2030', sSeason, sDay, sHour].value for sPE in ['sPEIMPCO', 'sPENAGAS', 'sPELNGAS', 'sPECROIL']
                       for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour) + \
                   sum(instance.vQPEImp[sPE, 'y2030', sSeason, sDay, sHour].value for sPE in ['sPEIMPCO', 'sPENAGAS', 'sPELNGAS', 'sPECROIL']
                       for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour)
    total_primary_energy = sum(instance.vQPEDom[sPE, 'y2030', sSeason, sDay, sHour].value for sPE in instance.sPE
                               for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour) + \
                           sum(instance.vQPEImp[sPE, 'y2030', sSeason, sDay, sHour].value for sPE in instance.sPE
                               for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour)
    fossil_fuel_use_2030 = total_fossil
    fossil_fuel_mix_2030 = total_fossil / total_primary_energy
    

    # Calculate RES deployment in 2030
    renewable_sources = ['sCEHYRURIV', 'sCEHYRSCAP', 'sCEMINIHYDR', 'sCEWINDON', 'sCEWINDOFF',
                        'sCESOPHVCEWT', 'sCESOPHVDIWOTIND', 'sCESOPHVDIWOTOTH', 'sCESOTELCE',
                     'sCEBIOELECE']

    total_renewable_generation = (sum(instance.vQCEPriOUT[sCE, sTE, 'y2030', sSeason, sDay, sHour].value 
                                     for (sCE, sTE) in instance.sQCEPriOUT if sCE in renewable_sources
                                     for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour)
                                + sum(instance.vQCESecOUT[sCE, sTE, 'y2030', sSeason, sDay, sHour].value
                                     for (sCE, sTE) in instance.sQCESecOUT if sCE in renewable_sources
                                     for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour))

    total_electricity_generation = (sum(instance.vQCEPriOUT[sCE, sTE, 'y2030', sSeason, sDay, sHour].value 
                                     for (sCE, sTE) in instance.sQCEPriOUT
                                     for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour)
                                + sum(instance.vQCESecOUT[sCE, sTE, 'y2030', sSeason, sDay, sHour].value
                                     for (sCE, sTE) in instance.sQCESecOUT
                                     for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour))

    RES_mix_2030 = total_renewable_generation / total_electricity_generation
    total_renewable_generation_2030 = total_renewable_generation
    total_electricity_generation_2030 = total_electricity_generation


    # Calculate electrification in 2030
    electricity_vectors = ['sTEELEIND', 'sTEELEOTH', 'sTEELETRA']

    electricity_consumed = sum(instance.vQSTInTE[sTE, sST, sES, sVin, 'y2030', sSeason, sDay, sHour].value
                                   for (sTE,sST,sES) in instance.sQTESTES_Ele for sVin in instance.sVin if (sVin,'y2030') in instance.sVinYear
                                   for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour)

    total_energy_consumed = sum(instance.vQSTInTE[sTE, sST, sES, sVin, 'y2030', sSeason, sDay, sHour].value
                                   for (sTE,sST,sES) in instance.sQTESTES for sVin in instance.sVin if (sVin,'y2030') in instance.sVinYear
                                   for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour)

    electrification_2030 = electricity_consumed / total_energy_consumed


        # Calculate EV adoption rate in 2030

    EV_vectors = ['sST_DSTRA_LNP_CABEV', 'sST_DSTRA_LNP_CAGSNPIHYB', 'sST_DSTRA_LNP_CADIEPIHYB']
    Car_vectors = ['sST_DSTRA_LNP_CAGSN', 'sST_DSTRA_LNP_CADST', 'sST_DSTRA_LNP_CACNG',
                   'sST_DSTRA_LNP_CALPG', 'sST_DSTRA_LNP_CABIOE85', 'sST_DSTRA_LNP_CABIOD85',
                   'sST_DSTRA_LNP_CAGSNPIHYB', 'sST_DSTRA_LNP_CADIEPIHYB', 'sST_DSTRA_LNP_CABEV']

    EV_2030 = sum(instance.vSTTotCap[sST, sVin, 'y2030'].value for sVin in instance.sVin if (sVin,'y2030') in instance.sVinYear
                      for sST in EV_vectors)
    Tot_Car_2030 = sum(instance.vSTTotCap[sST, sVin, 'y2030'].value for sVin in instance.sVin if (sVin,'y2030') in instance.sVinYear
                           for sST in Car_vectors)

    EV_2030 = sum(instance.vSTTotCap[sST, sVin, 'y2030'].value for sVin in instance.sVin if (sVin,'y2030') in instance.sVinYear
                      for sST in EV_vectors)
    Tot_Car_2030 = sum(instance.vSTTotCap[sST, sVin, 'y2030'].value for sVin in instance.sVin if (sVin,'y2030') in instance.sVinYear
                           for sST in Car_vectors)

        # Avoid division by zero
    if Tot_Car_2030 > 0:
            EV_rate_2030 = EV_2030 / Tot_Car_2030
    else:
            EV_rate_2030 = 0.0  # If no conventional vehicles are present, adoption rate is zero


    if Tot_Car_2030 > 0:
            EV_rate_2030 = EV_2030 / Tot_Car_2030
    else:
            EV_rate_2030 = 0.0  # If no conventional vehicles are present, adoption rate is zero



        # Energy transition cost
    acc_total_cost = sum(instance.vTotalCost[sYear].value for sYear in ['y2020', 'y2025', 'y2030'])
    energy_transition_cost = acc_total_cost

        # Total cost in 2030
    total_cost_2030 = instance.vTotalCost['y2030'].value

        # Energy dependency in 2030
    total_energy_imported = sum(instance.vQPEImp[sPE, 'y2030', sSeason, sDay, sHour].value for sPE in instance.sPE
                                    for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour)
    total_energy_domestic = sum(instance.vQPEDom[sPE, 'y2030', sSeason, sDay, sHour].value for sPE in instance.sPE
                                   for sSeason in instance.sSeason for sDay in instance.sDay for sHour in instance.sHour)
    energy_dependency_2030 = total_energy_imported / (total_energy_imported + total_energy_domestic)


        # Electricity affordability in 2030
        # Create lists to store dual values and consumptions
    dual_values = []
    consumptions = []

        # Iterate over the relevant elements of sTE

    for sSeason in instance.sSeason:
        for sDay in instance.sDay:
            for sHour in instance.sHour:
                try:
                    dual_value = instance.dual[instance.EQ_TEBalance['sTEELECE', 'y2030', sSeason, sDay, sHour]]
                    consumption = sum(instance.vQSTInTE[sTE, sST, sES, sVin, 'y2030', sSeason, sDay, sHour].value
                                      for (sTE, sST, sES) in instance.sQTESTES_Ele if sTE in ['sTEELECE']
                                      for sVin in instance.sVin if (sVin, 'y2030') in instance.sVinYear)
                    dual_values.append(dual_value)
                    consumptions.append(consumption)
                except KeyError:
                    print(f"No dual value for EQ_TEBalance['sTEELECE', 'y2030', {sSeason}, {sDay}, {sHour}]")

        # Calculate the weighted average of the dual values
    total_consumption = sum(consumptions)
    weighted_sum_dual_values = sum(dual * consumption for dual, consumption in zip(dual_values, consumptions))
    electricity_affordability_2030 = weighted_sum_dual_values / total_consumption if total_consumption != 0 else 0


    scenario_data = {
    'CO2_emissions_2030': CO2_emissions_2030,
    'CO2_budget_2030': CO2_budget_2030,
    'fossil_fuel_use_2030': fossil_fuel_use_2030,
    'fossil_fuel_mix_2030': fossil_fuel_mix_2030,
    'RES_mix_2030': RES_mix_2030,
    'total_renewable_generation_2030': total_renewable_generation_2030,
    'total_electricity_generation_2030': total_electricity_generation_2030,
    'electrification_2030': electrification_2030,
    'EV_rate_2030': EV_rate_2030,
    'EV_rate_2030': EV_rate_2030,
    'energy_transition_cost': energy_transition_cost,
    'total_cost_2030': total_cost_2030,
    'electricity_affordability_2030': electricity_affordability_2030,
    'energy_dependency_2030': energy_dependency_2030
    }

    # Add parameter values to the data
    for unc, level in zip(uncertainties, scenario):
        scenario_data[unc] = level

    # Add parameter values to the scenario data
    for param, values in param_dict.items():
        for value_dict in values:
            param_name = f"{param}_{value_dict['element1']}_{value_dict['element2']}_{value_dict['year']}"
            scenario_data[param_name] = value_dict['value']
            
    # Append the scenario data to the results list
    results_data.append(scenario_data)

    # Create a DataFrame for the results
    df_results = pd.DataFrame(results_data)

    # Save the results to a CSV file
    df_results.to_csv('../data/output/model_results.csv', index=False)


    end_time = time.time()
    iteration_time = end_time - start_time
    print(f"Iteration Time: {iteration_time} seconds")

# Create a DataFrame for the results
df_results = pd.DataFrame(results_data)

# Save the results to a CSV file
df_results.to_csv('../data/output/results_SD.csv', index=False)


In [ ]:
import os
Total_Cost_array = []
CO2_tot_array = []
EV_Rate_array = []
Tra_Emi_array = []

for i in range(len(df)):

    print(f"Iteration: {i}")

    instance.pEmiCO2Cost['y2030']=df['CO2_cost'][i]

    #instance.pPECost['sPEIMPCO', 'y2020']=df['Coal_cost'][i]
    instance.pPECost['sPEIMPCO', 'y2025']=df['Coal_cost'][i]
    instance.pPECost['sPEIMPCO', 'y2030']=df['Coal_cost'][i]

   # instance.pPECost['sPENAGAS', 'y2020']=df['NG_cost'][i]
    instance.pPECost['sPENAGAS', 'y2025']=df['NG_cost'][i]
    instance.pPECost['sPENAGAS', 'y2030']=df['NG_cost'][i]

    #instance.pPECost['sPELNGAS', 'y2020']=df['LNG_cost'][i]
    instance.pPECost['sPELNGAS', 'y2025']=df['LNG_cost'][i]
    instance.pPECost['sPELNGAS', 'y2030']=df['LNG_cost'][i]

    #instance.pPECost['sPECROIL', 'y2020']=df['Oil_cost'][i]
    instance.pPECost['sPECROIL', 'y2025']=df['Oil_cost'][i]
    instance.pPECost['sPECROIL', 'y2030']=df['Oil_cost'][i]

    instance.pCEMaxCap['sCEWINDON']=df['Wind_MaxRES'][i]

    instance.pCEMaxCap['sCESOPHVCEWT']=df['Solar_MaxRES'][i]

    instance.pCECapex['sCEWINDON', 'y2025']=df['Wind_capex'][i]
    instance.pCECapex['sCEWINDON', 'y2030']=df['Wind_capex'][i]

    instance.pCECapex['sCESOPHVCEWT', 'y2025']=df['Solar_capex'][i]
    instance.pCECapex['sCESOPHVCEWT', 'y2030']=df['Solar_capex'][i]

    #instance.pSTCapex['sST_DSTRA_LNP_CABEV', 'y2020']=df['EV_capex'][i]
    instance.pSTCapex['sST_DSTRA_LNP_CABEV', 'y2025']=df['EV_capex'][i]
    instance.pSTCapex['sST_DSTRA_LNP_CABEV', 'y2030']=df['EV_capex'][i]

    instance.pDC['sSD_DSTRA_PAS_URBN', 'sMD_TRA_PAS_URB']=df['Urban_mob_dem'][i]
    instance.pDC['sSD_DSTRA_PAS_URBN', 'sMD_TRA_PAS_RUR']=df['Urban_mob_dem'][i]

    instance.pAFTra['sST_DSTRA_LNP_CAGSN', 'sES_DSTRA_PAS_URBN', 'sSD_DSTRA_PAS_URBN']=df['Car_occup_rate'][i]
    instance.pAFTra['sST_DSTRA_LNP_CADST', 'sES_DSTRA_PAS_URBN', 'sSD_DSTRA_PAS_URBN']=df['Car_occup_rate'][i]
    instance.pAFTra['sST_DSTRA_LNP_CACNG', 'sES_DSTRA_PAS_URBN', 'sSD_DSTRA_PAS_URBN']=df['Car_occup_rate'][i]
    instance.pAFTra['sST_DSTRA_LNP_CALPG', 'sES_DSTRA_PAS_URBN', 'sSD_DSTRA_PAS_URBN']=df['Car_occup_rate'][i]
    instance.pAFTra['sST_DSTRA_LNP_CABIOE85', 'sES_DSTRA_PAS_URBN', 'sSD_DSTRA_PAS_URBN']=df['Car_occup_rate'][i]
    instance.pAFTra['sST_DSTRA_LNP_CABIOD85', 'sES_DSTRA_PAS_URBN', 'sSD_DSTRA_PAS_URBN']=df['Car_occup_rate'][i]
    instance.pAFTra['sST_DSTRA_LNP_CAGSNPIHYB', 'sES_DSTRA_PAS_URBN', 'sSD_DSTRA_PAS_URBN']=df['Car_occup_rate'][i]
    instance.pAFTra['sST_DSTRA_LNP_CADIEPIHYB', 'sES_DSTRA_PAS_URBN', 'sSD_DSTRA_PAS_URBN']=df['Car_occup_rate'][i]
    instance.pAFTra['sST_DSTRA_LNP_CABEV', 'sES_DSTRA_PAS_URBN', 'sSD_DSTRA_PAS_URBN']=df['Car_occup_rate'][i]

    instance.pDC['sSD_DSOTH_RES_HE', 'sMD_RES_DWE_ATL_SH']=df['Eff_buildings_HE'][i]
    instance.pDC['sSD_DSOTH_RES_HE', 'sMD_RES_DWE_ATL_BL']=df['Eff_buildings_HE'][i]
    instance.pDC['sSD_DSOTH_RES_HE', 'sMD_RES_DWE_CON_SH']=df['Eff_buildings_HE'][i]
    instance.pDC['sSD_DSOTH_RES_HE', 'sMD_RES_DWE_CON_BL']=df['Eff_buildings_HE'][i]            
    instance.pDC['sSD_DSOTH_RES_HE', 'sMD_RES_DWE_MED_SH']=df['Eff_buildings_HE'][i]
    instance.pDC['sSD_DSOTH_RES_HE', 'sMD_RES_DWE_MED_BL']=df['Eff_buildings_HE'][i]        

    instance.pDC['sSD_DSOTH_RES_LE', 'sMD_RES_DWE_ATL_SH']=df['Eff_buildings_LE'][i]
    instance.pDC['sSD_DSOTH_RES_LE', 'sMD_RES_DWE_ATL_BL']=df['Eff_buildings_LE'][i]
    instance.pDC['sSD_DSOTH_RES_LE', 'sMD_RES_DWE_CON_SH']=df['Eff_buildings_LE'][i]
    instance.pDC['sSD_DSOTH_RES_LE', 'sMD_RES_DWE_CON_BL']=df['Eff_buildings_LE'][i]            
    instance.pDC['sSD_DSOTH_RES_LE', 'sMD_RES_DWE_MED_SH']=df['Eff_buildings_LE'][i]
    instance.pDC['sSD_DSOTH_RES_LE', 'sMD_RES_DWE_MED_BL']=df['Eff_buildings_LE'][i]        
					
    solver = pyo.SolverFactory('gurobi')

    solver_results = solver.solve(instance, keepfiles=False, tee=True)
    path        = "../data/input/openMASTER_Data.xlsx"
    base_folder = '../data/tmp/output'
    output_path = os.path.join(base_folder, str(i))
    sheetname   = "Output"

    d_vars_from_instance = openMASTER.export_model_to_csv(path, output_path, sheetname, instance)

    #Resultados
    total_cost=instance.vSysCost.value
    CO2_tot=instance.vEmiCO2Tot['y2030'].value

    data_2=[]
    # Iterate over all combinations of sST, sVinYear
    for sST in instance.sST:
        for sVinYear in instance.sVinYear:
            # Extract sVin and sYear from the related set
            sVin, sYear = sVinYear

            # Append a dictionary with the relevant values
            data_2.append({
                'sST': sST,
                'sVin': sVin,
                'sYear': sYear,
                'vSTTotCap': instance.vSTTotCap[sST, sVin, sYear].value
            })

    # Create a dataframe from the list of dictionaries
    df_ST = pd.DataFrame(data_2)
    
    EV_Techs=['sST_DSTRA_LNP_CABEV', 'sST_DSTRA_LNP_CAGSNPIHYB', 'sST_DSTRA_LNP_CADIEPIHYB']
    Car_Techs=['sST_DSTRA_LNP_CAGSN', 'sST_DSTRA_LNP_CADST', 'sST_DSTRA_LNP_CACNG', 'sST_DSTRA_LNP_CALPG', 'sST_DSTRA_LNP_CABIOE85', 'sST_DSTRA_LNP_CABIOD85', 'sST_DSTRA_LNP_CAGSNPIHYB', 'sST_DSTRA_LNP_CADIEPIHYB', 'sST_DSTRA_LNP_CABEV']
    EV_2030 = df_ST[(df_ST['sYear'] == 'y2030') & (df_ST['sST'].isin(EV_Techs))]
    Tot_Car_2030=df_ST[(df_ST['sYear'] == 'y2030') & (df_ST['sST'].isin(Car_Techs))]
    EV_Rate=sum(EV_2030['vSTTotCap'])/sum(Tot_Car_2030['vSTTotCap'])

    data_3=[]
    # Iterate over all combinations of sST, sVinYear
    for sQSTESSD in instance.sQSTESSD:
        sST, sTE, sSD = sQSTESSD
        for sYear in instance.sYear:
            data_3.append({
                'sST': sST,
                'sTE': sTE,
                'sYear': sYear,      
                'vEmiCO2ST': instance.vEmiCO2ST[sST, sTE, sYear].value    
            })

    df_Emi = pd.DataFrame(data_3)
    df_Emi.head()
    Tra_Tech= ["sST_DSTRA_LNP_CAGSN", "sST_DSTRA_LNP_CADST", "sST_DSTRA_LNP_CACNG", "sST_DSTRA_LNP_CALPG", "sST_DSTRA_LNP_CABIOE85", "sST_DSTRA_LNP_CABIOD85", "sST_DSTRA_LNP_CAGSNPIHYB", "sST_DSTRA_LNP_CADIEPIHYB", "sST_DSTRA_LNP_CABEV", "sST_DSTRA_AIR_AIRPLANEKERO", "sST_DSTRA_SEA_SHIPDIE", "sST_DSTRA_SEA_SHIPFOI", "sST_DSTRA_LNP_MOPGSN", "sST_DSTRA_LNP_BUSDST", "sST_DSTRA_LNP_BUSELE", "sST_DSTRA_LNP_BUSCNG", "sST_DSTRA_LNP_COADST", "sST_DSTRA_LNP_COACNG", "sST_DSTRA_RAIL_URBAN", "sST_DSTRA_RAIL_INTER", "sST_DSTRA_AIRF_AIRPLANEKERO", "sST_DSTRA_LNF_VANSMGSN", "sST_DSTRA_LNF_VANSMELE", "sST_DSTRA_LNF_VANSMBIO", "sST_DSTRA_LNF_VANSMCNG", "sST_DSTRA_LNF_VANSMLPG", "sST_DSTRA_LNF_TRUBIDIE", "sST_DSTRA_SEAF_SHIPDIE"]
    Emi_Tra = df_Emi[(df_Emi['sYear'] == 'y2030') & (df_ST['sST'].isin(Tra_Tech))]
    Emi_Tra_value=sum(Emi_Tra['vEmiCO2ST'])

    Total_Cost_array.append(total_cost)
    CO2_tot_array.append(CO2_tot)
    EV_Rate_array.append(EV_Rate)
    Tra_Emi_array.append(Emi_Tra_value)
    print(total_cost)
    print(CO2_tot)
    print(EV_Rate)
    print(Emi_Tra_value)


In [ ]:
df['Total_Cost'] = Total_Cost_array
df['CO2_tot'] = CO2_tot_array
df['EV_Rate'] = EV_Rate_array
df['Emi_Tra']=Tra_Emi_array

In [ ]:
df.to_csv("results_SD.csv")

In [ ]:
import matplotlib.pyplot as plt
plt.hist(df['CO2_tot'], color='lightgreen', ec='black', bins=15)
plt.xlabel("Values")
plt.ylabel("Frequency")
plt.title("Distribution Plot using Matplotlib")
plt.show()
32/26